## Notebook to test for Random Things

In [7]:
import numpy as np
import pandas as pd
from mlfinlab.online_portfolio_selection import *
#from mlfinlab.online_portfolio_selection.BAH import BAH


/Users/alexkwon/opt/anaconda3/envs/ht/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/alexkwon/opt/anaconda3/envs/ht/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)




Support us on Patreon: https://www.patreon.com/HudsonThames

MlFinLab needs you! We need your help for us to keep on maintaining and implementing academic research based on 
financial machine learning (for open-source). In order for us to continue we need to raise $4000 of monthly donations
via Patreon - by December 2020. If we can't reach our goal, we will need to adopt more of a paid for service. We thought
that the best and least impactful course of action (should we not reach our goal) is to leave the package as open-source
but to make the documentation (ReadTheDocs) a paid for service. This is the ultimate litmus test, if the package is a 
value add, then we need the community to help us keep it going.

Our road map for 2020 is to implement the text book: Machine Learning for Asset Managers by Marcos Lopez de Prado, 
as well as a few papers from the Journal of Financial Data Science. We are hiring a full time developer for 3 months 
to help us reach our goals. The money that you

In [8]:
stock_prices = pd.read_csv('../tests/test_data/stock_prices.csv', parse_dates=True, index_col='Date')
stock_prices = stock_prices.dropna(axis=1)
asset_name = list(stock_prices.columns)

In [13]:
stock_prices.pct_change().fillna(0) + 1

,EEM,EWG,TIP,EWJ,EFA,IEF,EWQ,EWU,XLB,XLE,...,XLU,EPP,FXI,VGK,VPL,SPY,TLT,BND,CSJ,DIA
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2008-01-03,1.008997,0.997174,1.003376,1.003779,1.001662,1.002054,0.999473,1.006281,1.018160,1.011824,...,0.998574,1.002345,0.993153,1.003623,1.002823,0.999517,0.998623,1.001293,1.001183,1.000842
2008-01-04,0.969963,0.980448,0.999720,0.974398,0.977281,1.002619,0.975475,0.976280,0.969798,0.963451,...,1.007614,0.971861,0.980875,0.975668,0.974370,0.975494,1.000212,1.001162,1.001281,0.980343
2008-01-07,1.007327,1.000867,0.999813,0.993818,1.001045,1.001817,1.007299,1.002984,0.986268,0.996129,...,1.020779,1.011368,1.028971,0.999589,0.998176,0.999151,1.004349,1.000258,1.000689,0.999142
2008-01-08,0.992246,0.993070,1.000748,0.997667,0.994390,1.002154,0.992217,0.985975,0.984585,0.982513,...,1.000231,0.983140,0.985922,0.992461,0.995583,0.983852,0.998838,1.001031,1.000197,0.982742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-27,0.987136,0.975734,1.002937,0.994709,0.980805,1.008645,0.976712,0.960192,0.966319,0.967954,...,1.012299,0.979095,0.996614,0.973921,0.990431,0.982090,1.024967,1.005018,0.999339,0.986062
2016-06-28,1.029476,1.020506,1.003445,1.015071,1.026221,0.999823,1.030388,1.042888,1.008545,1.027847,...,1.003723,1.025676,1.024089,1.032779,1.020624,1.018036,1.002084,1.001189,1.000945,1.015304
2016-06-29,1.025316,1.013681,0.999399,1.013100,1.018330,0.995935,1.017695,1.026045,1.016945,1.018964,...,1.002733,1.015547,1.018094,1.020340,1.013472,1.017028,0.992327,0.999644,1.000472,1.016224
